In [ ]:
import spotipy as sp
import spotipy.util as util
import pandas as pd
import os
import time
import requests
from bs4 import BeautifulSoup as bs
import datetime
from datetime import timedelta, date
import glob
from os import listdir
import numpy as np
import pymongo
import csv
import json
import sys, getopt, pprint
from pymongo import MongoClient
import types
import itertools
from itertools import zip_longest

In [ ]:
def current_user_recently_played(self, limit=50):
    return self._get('me/player/recently-played', limit=limit)

In [ ]:
scope="user-read-recently-played user-read-private user-top-read user-read-currently-playing"

In [ ]:
if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print ("Usage: %s username" % (sys.argv[0],))
    sys.exit()

In [ ]:
token = util.prompt_for_user_token(username,scope,
                           client_id='e67d7b1bc1ee460bbef7484ee3f601ef',
                           client_secret='ba0087b5f4ab4fb8ab88be1850854538',
                           redirect_uri='https://google.com')

In [ ]:
spotify = spotipy.Spotify(auth=token)
spotify.current_user_recently_played = types.MethodType(current_user_recently_played, spotify)



In [ ]:
dave = spotify.current_user_recently_played(limit=50)
out_file = open("dave.json","w")
out_file.write(json.dumps(dave, sort_keys=True, indent=2))
out_file.close()

In [ ]:
#print(json.dumps(dave, sort_keys=True, indent=2))

In [ ]:
Tracks = []

for i in range(50):
   Tracks.append(dave['items'][i]['track']['name'])

TrackID = []

for i in range(50):
   TrackID.append(dave['items'][i]['track']['id'])

In [ ]:
Artists = []

for i in range(50):
   Artists.append(dave['items'][i]['track']['artists'][0]['name'])

In [ ]:
Albums = []


for i in range(50):
   Albums.append(dave['items'][i]['track']['album']['name'])

In [ ]:
for (a, b, c, d) in itertools.zip_longest(TrackID, Tracks, Artists, Albums): 
    print (a, b, c,d ) 

In [ ]:
dave_df = pd.DataFrame(np.column_stack([TrackID, Tracks, Artists, Albums]), 
                               columns=['Track ID','Tracks', 'Artists', 'Albums'])

In [ ]:
dave_df.head(5)

In [ ]:
TrackID

In [50]:
features = []
for i in range(0,len(TrackID),50):
    audio_features = spotify.audio_features(TrackID[i:i+50])
    for track in audio_features:
        features.append(track)
        features[-1]['target'] = 1

In [71]:
features

[{'danceability': 0.376,
  'energy': 0.43,
  'key': 0,
  'loudness': -6.585,
  'mode': 0,
  'speechiness': 0.0328,
  'acousticness': 0.0902,
  'instrumentalness': 0,
  'liveness': 0.0912,
  'valence': 0.129,
  'tempo': 77.599,
  'type': 'audio_features',
  'id': '4Yxc55NX3tAXC2mHRAhtcW',
  'uri': 'spotify:track:4Yxc55NX3tAXC2mHRAhtcW',
  'track_href': 'https://api.spotify.com/v1/tracks/4Yxc55NX3tAXC2mHRAhtcW',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4Yxc55NX3tAXC2mHRAhtcW',
  'duration_ms': 177323,
  'time_signature': 3,
  'target': 1},
 {'danceability': 0.598,
  'energy': 0.427,
  'key': 7,
  'loudness': -8.764,
  'mode': 0,
  'speechiness': 0.0317,
  'acousticness': 0.0546,
  'instrumentalness': 5.83e-06,
  'liveness': 0.21,
  'valence': 0.0605,
  'tempo': 76.469,
  'type': 'audio_features',
  'id': '3eekarcy7kvN4yt5ZFzltW',
  'uri': 'spotify:track:3eekarcy7kvN4yt5ZFzltW',
  'track_href': 'https://api.spotify.com/v1/tracks/3eekarcy7kvN4yt5ZFzltW',
  'analysis_url

In [60]:
audiofeat_df = pd.DataFrame(features)

In [61]:
audiofeat_df.drop(audiofeat_df.loc[:,'uri':'target'], inplace = True, axis = 1)


In [64]:
del audiofeat_df['type']

In [66]:
audiofeat_df.head(1)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id
0,0.376,0.43,0,-6.585,0,0.0328,0.0902,0.0,0.0912,0.129,77.599,4Yxc55NX3tAXC2mHRAhtcW


In [67]:
DaveDF = dave_df.join(audiofeat_df)

In [69]:
del DaveDF['id']